In [271]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [272]:
print_Debt = True

In [273]:
quantum_columns = str(export_from_RISKCUSTOM("""select *from "RISKACCESS"."quantumDebt" fetch first 1 rows only""").columns.tolist()).replace('\'','\"')[1:-1]
dates = ['31/08/24', '30/09/24']
dates_SQL = str([f'TO_DATE(\'{item}\', \'DD/MM/YY\')' for item in dates])[1:-1].replace('\"','')
dates_SQL

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


"TO_DATE('31/08/24', 'DD/MM/YY'), TO_DATE('30/09/24', 'DD/MM/YY')"

Долг источник:

In [ ]:
# quantum debt
query = f"""
with table_1 as
(select 
{quantum_columns},
--"reportDate","effectiveFrom","entity","dealNo","holding","counterparty","amount","eventCurrency","amountUSD","actionDate","dealClass1","dealClass2","instrumentOwner","dealSet","facility", 
MAX("effectiveFrom") OVER (partition BY "reportDate") as "max_effectiveFrom"
from "RISKACCESS"."quantumDebt" 
where "reportDate" in ({dates_SQL})
--fetch first 10 rows only
)
select * from table_1
where "effectiveFrom" = table_1."max_effectiveFrom"
--fetch first 10 rows only
"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.head(2)

,##batch_id,##deleted,##inserted,##origin_name,flag,actionDate,amount,eventCurrency,amountUSD,currentRate,...,debtRelated,reportDate,eventNumber,entitySAPBUCode,counterpartySAPID,counterpartySWIFTBICCode,effectiveFrom,xlsDate,loadEventId,max_effectiveFrom
0,41600664@WHPROD,N,2024-10-31 23:15:35,QUANTUM,-,2040-10-10,-71669.98,RUB,-785.968803,3.0,...,Y,2024-08-31,235578,Y300,1000147688,None,2024-10-31,2024-10-31 23:14:42,285089,2024-10-31
1,41600664@WHPROD,N,2024-10-31 23:15:35,QUANTUM,-,2041-01-10,-72211.92,RUB,-791.911987,3.0,...,Y,2024-08-31,235579,Y300,1000147688,None,2024-10-31,2024-10-31 23:14:42,285089,2024-10-31


In [314]:
data_Debt_export.loc[data_Debt_export.dealNo==4816,['reportDate', 'effectiveFrom','entity','counterparty','amount']]

,reportDate,effectiveFrom,entity,counterparty,amount
6171,2024-08-31,2024-10-31,KZUDB,EC-Karatau (EC),4.500000e+08
6172,2024-08-31,2024-10-31,KZUDB,EC-Karatau (EC),5.000000e+08
6173,2024-08-31,2024-10-31,KZUDB,EC-Karatau (EC),4.000000e+08
6402,2024-08-31,2024-10-31,KZUDB,EC-Karatau (EC),3.723584e+09
13518,2024-09-30,2024-11-05,KZUDB,EC-Karatau (EC),4.000000e+08
13519,2024-09-30,2024-11-05,KZUDB,EC-Karatau (EC),3.723584e+09


In [318]:
query = f"""
select "reportDate", "effectiveFrom","entity","counterparty","amount" from 
"RISKACCESS"."quantumDebt"
where "reportDate" in ({dates_SQL})
and "dealNo" = '4816'
--fetch first 10 rows only
"""
data = export_from_RISKCUSTOM(query)
data

,reportDate,effectiveFrom,entity,counterparty,amount
0,2024-08-31,2024-10-20,KZUDB,EC-Karatau (EC),4.500000e+08
1,2024-08-31,2024-10-19,KZUDB,EC-Karatau (EC),4.500000e+08
2,2024-08-31,2024-10-19,KZUDB,EC-Karatau (EC),5.000000e+08
3,2024-08-31,2024-10-20,KZUDB,EC-Karatau (EC),5.000000e+08
4,2024-09-30,2024-11-05,KZUDB,EC-Karatau (EC),4.000000e+08
...,...,...,...,...,...
196,2024-09-30,2024-10-14,KZUDB,EuroChem-Karatau (EC),4.123584e+09
197,2024-08-31,2024-10-08,KZUDB,EuroChem-Karatau (EC),4.123584e+09
198,2024-09-30,2024-10-08,KZUDB,EuroChem-Karatau (EC),4.123584e+09
199,2024-09-30,2024-10-13,KZUDB,EuroChem-Karatau (EC),4.123584e+09


In [ ]:
# данные заносятся ч задержкой вручную. Нам выходит настроенный отчет за определенную дату
# дата формирования отчета - репорт дэйт
# дата когда он был физически сформирован - эффектив фром
# на эффектив фром выгружается цельный отчет - состояние системы

In [275]:
assert len(data_Debt_export[data_Debt_export.entity.isna()]) == 0 # data quality
data_Debt_export.dealNo = data_Debt_export.dealNo.astype(int)
data_Debt_work = data_Debt_export.copy() # copy data
# merge
data_Debt_work['entity_code'] = merge_Mapping(data_Debt_work, col='entity')
data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity_code'] = data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity']
data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity_code', merge_col='ocpSegment')

# Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet
# для каждой даты
full_r_data = pd.DataFrame()
for r_date in data_Debt_work.reportDate.unique():
    r_date_data = data_Debt_work[data_Debt_work.reportDate==r_date]
    # Выбор нужного разреза данных
    deals_data_Bonds = r_date_data.loc[r_date_data.dealSet.str.contains('Bond'), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

    Bonds_list = deals_data_Bonds.dealSet.unique()

    deals_data_Bonds_work = pd.DataFrame()
    for Bond in Bonds_list:
        try:
            Bond_data = deals_data_Bonds[deals_data_Bonds.dealSet==Bond] # Отбор данных одного Bond
            Bond_data_traded = Bond_data[Bond_data.instrumentOwner=='EC_TRADED_BONDS']
            Bond_data_issued = Bond_data[Bond_data.instrumentOwner=='EC_ISSUED_BONDS']
            if len(Bond_data_issued)>1:
                Bond_data_issued['index_copy'] = Bond_data_issued.index # copy index
                # groupby
                Bond_data_issued_pivot = Bond_data_issued.groupby(as_index=False, dropna=False, by=['entity_code',	'Entity_group',	'counterparty', 'instrumentOwner',	'dealSet',	'reportDate']).agg({'amount':'sum', 'amountUSD':'sum', 'index_copy':'min'})
                Bond_data_issued_pivot.index = [int(Bond_data_issued_pivot.index_copy.item())] # index on min in group
                Bond_data_issued_pivot.drop(columns='index_copy') # drop index copy col
                Bond_data = pd.concat([Bond_data_traded, Bond_data_issued_pivot])
            assert len(Bond_data_issued) != 0

            columns_list = ['amount', 'amountUSD']
            for column in columns_list:
                Traded_data_sum = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_TRADED_BONDS', column].sum())
                Issued_data = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', column].item())
                # Внесение нового значения amount в issued bonds
                Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
            # Внесение нового значения counterparty в traded bonds
            Issied_data_entity = Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', 'entity_code'].squeeze()
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
            
            deals_data_Bonds_work = pd.concat([deals_data_Bonds_work, Bond_data])
            # deals_data_Bonds[deals_data_Bonds.dealSet==Bond] = Bond_data
        except:
            continue

    index_list = deals_data_Bonds_work.index.to_list()
    r_date_data.loc[(r_date_data.dealSet.str.contains('Bond'))&(r_date_data.index.isin(index_list)), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] = deals_data_Bonds_work
    r_date_data.loc[(r_date_data.dealSet.str.contains('Bond'))&(~r_date_data.index.isin(index_list)), 'entity_code'] = np.nan
    r_date_data = r_date_data[~r_date_data.entity_code.isna()]
    # Запись выполненного цикла в в основные данные
    full_r_data = pd.concat([full_r_data,r_date_data])
data_Debt_work = full_r_data

data_Debt_work = Period(data_Debt_work, day_for_count='reportDate', col_with_date='actionDate', day_is_col=True).reset_index(drop=True)

data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty_name'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
data_Debt_work.loc[~data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty_name'] = data_Debt_work.loc[~data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty']
# Обрезка " (EC)" в каждой строке если он есть

assert len(data_Debt_export[data_Debt_export.counterparty.isna()])==0
data_Debt_work['cty_Code'] = merge_Mapping(data_Debt_work, col='counterparty_name')
data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='cty_Code', merge_col='ocpSegment')
# Merge counterparty с Mapping и SalesUnits

data_Debt_work['Source'] = 'Quantum'

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SGC' 'SGC' 'SGC' ... 'SAMSALES' 'SAMSALES' 'SAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\4040821026.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [276]:
# instrumentOwner_list = ['EC_ISSUED_BONDS', 'EC_TRADED_BONDS']
# deals_data_Bonds[(deals_data_Bonds.dealSet==Bonds_list[0])&(deals_data_Bonds.reportDate==dates[0])&(deals_data_Bonds.instrumentOwner==instrumentOwner_list[1])]
# data_Debt_work.loc[(data_Debt_work.dealSet==Bonds_list[0])&(data_Debt_work.reportDate==dates[0])&(data_Debt_work.instrumentOwner==instrumentOwner_list[1]), ['entity_code','Entity_group','counterparty','amount','amountUSD','instrumentOwner','dealSet','reportDate']]
# data_Debt_export.loc[(data_Debt_export.instrumentOwner=='EC_TRADED_BONDS')&(data_Debt_export.dealSet=='SUEK EuBond 2021 $500m B'), ['entity','counterparty','instrumentOwner','dealSet','reportDate']]

Деривативы:

In [277]:
query = f"""select * 
from "RISKACCESS"."quantumForwards" 
where "positionDate" in ({dates_SQL})"""
data_Forwards_export = export_from_RISKCUSTOM(query)

query = f"""select * 
from "RISKACCESS"."quantumSwaps" 
where "positionDate" in ({dates_SQL})"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
data_Deriv_qu.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or

,##batch_id,##deleted,##inserted,##origin_name,adjustedProfitLoss,baseCurrency,baseType,checkSettlementDateAlt,contractRate,counterparty,...,recMargin,recPrincipalFreq,recRateBasisCode,recRateBasisFreq,recRateBasisName,recRateFormula,recRatesetFreq,shortInterestRate,shortNPVOfAdjustedProfitLoss,startdate
0,40656537@WHPROD,N,2024-10-01 01:00:17,QUANTUM,344.36,USD,CONTRACT,Y,1.367691,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,40656537@WHPROD,N,2024-10-01 01:00:17,QUANTUM,-2003.71,USD,CONTRACT,Y,1.359600,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [278]:
data_Deriv_qu.dealNo = data_Deriv_qu.dealNo.astype(int)
data_Deriv_qu_work = data_Deriv_qu.copy()
# проверка вхождения в debt
data_Deriv_qu_work['in_debt'] = data_Deriv_qu.dealNo.isin(data_Debt_export.dealNo.unique().tolist())
# data_Deriv_qu_work = data_Deriv_qu_work[data_Deriv_qu_work.in_debt==False]
# melt data
melt_data = pd.melt(data_Deriv_qu_work, id_vars=['positionDate','holdingEntity','entity','counterparty','dealNo','payFaceValue','recFaceValue','maturityDate','in_debt'], value_vars=['payFXCurrency','recFXCurrency'], value_name='Currency').sort_values('dealNo').rename(columns={'recFaceValue':'amount'})
melt_data.loc[melt_data.variable=='payFXCurrency', 'amount'] = melt_data.loc[melt_data.variable=='payFXCurrency', 'payFaceValue'] * -1
melt_data = melt_data.drop(columns=['payFaceValue','variable']).reset_index(drop=True)
# rensme кривых названий
melt_data['entity'] = merge_Mapping(melt_data, 'entity')
melt_data['holdingEntity'] = merge_SalesUnits(melt_data, 'entity', merge_col='holding')
melt_data['Entity_group'] = merge_SalesUnits(melt_data, col='entity', merge_col='ocpSegment')

data_Deriv_qu.shape[0], data_Deriv_qu_work.shape[0]

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'SUEK' 'SUEK' 'SUEK' 'SUEK'
 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK'
 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK' 'SUEK'
 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM'
 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM'
 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM'
 'EUROCHEM' 'EUROCHEM' 'EUROCHEM' 'EUROCHEM

(54, 54)

In [279]:
# deriv_in_debt_dealno_list = [x for x in Deriv_dealNo if x in Debt_dealNo]
# deriv_in_debt_dealno_list.sort()

# deriv_OCP_not_in_melt_list =  list(set([x for x in data_Deriv_OCP.recordId.tolist() if x not in melt_data.dealNo.tolist()]))
# deriv_OCP_not_in_melt_list.sort()


In [280]:
# with pd.ExcelWriter(f'{str(date.today())}_to_IT_quDeriv.xlsx', engine='openpyxl') as writer:
#     pd.DataFrame({'recordId\dealNo_in_quDebt':deriv_in_debt_dealno_list}, index = range(len(deriv_in_debt_dealno_list))).to_excel(writer, index=False, sheet_name='records_id_list')
#     data_Debt_export[data_Debt_export.dealNo.isin(deriv_in_debt_dealno_list)].to_excel(writer, index=False, sheet_name='quDebt_data')
#     Deriv_merge.to_excel(writer, index=False, sheet_name='merge_qudebt_to_OCP')

Долг OCP

In [281]:
query = f"""
select * 
from xxwh."unifiedOcp"
where "sourceTable" in ('quantumDebt_bonds','quantumDebt')
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
data_OCP_export = export_from_WHWEEK(query)
data_OCP_export.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,41758828@WHWEEK,N,2024-11-06 10:59:32,WHFLEX,2024-08-31,M,QUANTUM,quantumDebt,1456,On-balance,...,3M-6M,USD,-4.787743e+07,USD,1,-4.787743e+07,1.000000,-4.787743e+07,1L00,None
1,41758828@WHWEEK,N,2024-11-06 10:59:32,WHFLEX,2024-08-31,M,QUANTUM,quantumDebt,1456,On-balance,...,3M-6M,RUB,3.000000e+09,RUB,1,3.000000e+09,0.011054,3.316189e+07,1L00,None


In [307]:
data_OCP_export[data_OCP_export.recordId==4816].groupby(as_index=False, by=['reportDate','recordId'])['volumeEffectiveCurrency'].agg(sum)
data_OCP_export.loc[(data_OCP_export.recordId==4816)&(data_Debt_export.reportDate=='2024-08-31'), ['buCode', 'buName','counterpartyName','volumeEffectiveCurrency']].reset_index()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\2847131824.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_OCP_export[data_OCP_export.recordId==4816].groupby(as_index=False, by=['reportDate','recordId'])['volumeEffectiveCurrency'].agg(sum)


,index,buCode,buName,counterpartyName,volumeEffectiveCurrency
0,1134,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EC-Karatau (EC),4.500000e+08
1,1135,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EC-Karatau (EC),3.723584e+09
2,1136,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EC-Karatau (EC),4.000000e+08
3,1137,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EC-Karatau (EC),5.000000e+08
4,1138,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),4.500000e+08
5,1139,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),4.123584e+09
6,1140,KZUDB,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),5.000000e+08
7,1141,KZKRT,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),-4.500000e+08
8,1142,KZKRT,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),-4.123584e+09
9,1147,KZKRT,"ТОО ""ЕвроХим-Удобрения""",EuroChem-Karatau (EC),-5.000000e+08


In [308]:
data_Debt_export.columns

Index(['##batch_id', '##deleted', '##inserted', '##origin_name', 'flag',
       'actionDate', 'amount', 'eventCurrency', 'amountUSD', 'currentRate',
       'annualInterestUSD', 'amountUSDForMaturity', 'dealNo', 'dealClass1',
       'dealClass2', 'instrumentOwner', 'dealSet', 'facility', 'facilityAlt',
       'loanType', 'facilityAlt2', 'dealTicket', 'holding', 'entity',
       'counterparty', 'counterpartyAlt', 'legDirection', 'dealInternalL0',
       'dealInternalL1', 'issuedLoan', 'year', 'month', 'monthOrdinal',
       'transactionType', 'instrument', 'mmType', 'componentClass',
       'component', 'fixFloat', 'rateFormula', 'basis', 'basisFrequency',
       'currentBaseRate', 'currentMargin', 'rateSetDate', 'cODType',
       'cpartyCountry', 'cpartyRelationL1', 'cpartyType',
       'cpartyDealInternalL0', 'dealExternalL0', 'dealDate', 'settlementDate',
       'currentMaturityDate', 'effectiveDate', 'offerDate', 'entityCountry',
       'entityOwner', 'fxMultiplyToUSD', 'maxFXRateDat

In [309]:
data_Debt_export[data_Debt_export.dealNo==4816].groupby(as_index=False, by=['reportDate','dealNo'])['amount'].agg(sum)
data_Debt_export.loc[(data_Debt_export.dealNo==4816)&(data_Debt_export.reportDate=='2024-08-31'), ['entity','counterparty','amount']]

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\258853489.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_Debt_export[data_Debt_export.dealNo==4816].groupby(as_index=False, by=['reportDate','dealNo'])['amount'].agg(sum)


,entity,counterparty,amount
6171,KZUDB,EC-Karatau (EC),4.500000e+08
6172,KZUDB,EC-Karatau (EC),5.000000e+08
6173,KZUDB,EC-Karatau (EC),4.000000e+08
6402,KZUDB,EC-Karatau (EC),3.723584e+09


In [284]:
### Compairing Debt
data_OCP_export.recordId = data_OCP_export.recordId.astype(int)
# rename
rename_dict = {'ocpSegment':'Entity_group','effectiveCurrency':'eventCurrency','salesUnitsId':'entity_code','volumeEffectiveCurrency':'amount_OCP','volumeUsd':'amountUSD_OCP','counterpartyName':'counterparty','recordId':'dealNo'}
data_OCP_work = data_OCP_export.copy()
data_Debt_work.holding = data_Debt_work.holding.replace({'EuroChem':'EUROCHEM'})
# cty code without xxtsu
data_OCP_work['cty_Code'] = merge_Mapping(data_OCP_work, 'introgroupCounterpartyCode')
data_OCP_work['Counterparty_Group'] = merge_SalesUnits(data_OCP_work, 'cty_Code', merge_col='ocpSegment')

data_OCP_work = data_OCP_work.rename(columns=rename_dict)
# first compairing
# Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency']
# Debt_first = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
# Debt_first.holding = Debt_first.holding.replace({'EuroChem':'EUROCHEM'})
# OCP_first = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# # merge
# First_merge = Debt_first.merge(OCP_first, how='outer', on=Debt_index_list)
# First_merge = is_approximately_equal_for_cols(First_merge, 'amount', 'amount_OCP')
# First_merge_not_equal = First_merge[First_merge.amount!=First_merge.amount_OCP]

# second compairing
# First_merge_not_equal = concat_columns(First_merge_not_equal, columns=Debt_index_list)
# concat_list = First_merge_not_equal.concat_columns.tolist()
# data_Debt_work = concat_columns(data_Debt_work, columns=Debt_index_list)
# data_OCP_work = concat_columns(data_OCP_work, columns=Debt_index_list)

Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency', 'Counterparty_Group','cty_Code','dealNo']
# data_Debt_work.dealNo,data_OCP_work.dealNo = data_Debt_work.dealNo.astype(int),data_OCP_work.dealNo.astype(int)
# Debt_second = data_Debt_work[data_Debt_work.concat_columns.isin(concat_list)].groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
# OCP_second = data_OCP_work[data_OCP_work.concat_columns.isin(concat_list)].groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
Debt_second = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
OCP_second = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# merge
Second_merge = Debt_second.merge(OCP_second, how='outer', on=Debt_index_list)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT'
 'RUFERT' 'SPV' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT'
 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'EURSALES' 'EURSALES' 'TRADING'
 'TRADING' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT'
 'RUFERT' 'SPV' 'SPV' 'TRADING' 'TRADING' 'EURSALES' 'EURSALES' 'TRADING'
 'TRADING' 'TRADING' 'EURSALES' 'EURSALES' 'NAMSALES' 'NAMSALES' 'TRADING'
 'TRADING' 'TRADING' 'SA

In [285]:
# если чего-то нет в amount, проверить есть ли тот же объем с обратным котрагентом
Second_merge_work = Second_merge.copy()
Second_merge_work['abs_amount_OCP'] = Second_merge_work.amount_OCP.abs()
Second_merge_work['index_copy'] = Second_merge_work.index
Second_merge_work = concat_columns(Second_merge_work, ['dealNo', 'eventCurrency','abs_amount_OCP', 'entity_code','cty_Code']).rename(columns={'concat_columns':'E_C'})
data_amount_isna = Second_merge_work[Second_merge_work.amount.isna()]
data_amount_isna = concat_columns(data_amount_isna, ['dealNo', 'eventCurrency', 'abs_amount_OCP', 'cty_Code', 'entity_code']).rename(columns={'concat_columns':'C_E'})
drop_index = data_amount_isna[data_amount_isna.C_E.isin(Second_merge_work.E_C.tolist())].index_copy.tolist() # берем те строки, лля которых есть обратные операции
Second_merge_work_not_duples = Second_merge_work[~Second_merge_work.index_copy.isin(drop_index)] # удаляем обатные операции
# сравниваем значения на равенство
Second_merge_work_not_duples.amount = Second_merge_work_not_duples.amount.fillna(0)
Second_merge_work_not_duples = is_approximately_equal_for_cols(Second_merge_work_not_duples, 'amount', 'amount_OCP')
# поделить суммы и если КФ равен 2, то проигнорировать ее
Second_merge_work_not_duples['CF_del_amounts'] = Second_merge_work_not_duples.amount_OCP/Second_merge_work_not_duples.amount
Second_merge_work_not_duples.loc[Second_merge_work_not_duples.CF_del_amounts==2, 'Diff__more_0.4'] = False
Second_merge_work_not_duples = Second_merge_work_not_duples.drop(columns=['index_copy','E_C'])

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concat_columns'] = df[columns].astype(str).apply(lambda row: sign.join(row.values.astype(str)), axis=1)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17868\763116701.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Second_merge_work_not_duples.amount = Second_merge_work_not_duples.amount.fillna(0)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.p

In [286]:
# точно ли там где amount равен нулю, в исходных данных нет информации по данному номеру сделки
delano_list = Second_merge_not_duples.loc[Second_merge_not_duples['Diff__more_0.4']==True, 'dealNo'].tolist()
manual_check_dealno_list = list(set(data_Debt_export.dealNo.tolist()).intersection(set(delano_list)))

Деривативы OCP

In [ ]:
query = f"""
select * 
from xxwh."unifiedOcp"
where "sourceTable" in ('quantumForwards', 'quantumSwaps')
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
# quantumSwaps
data_Deriv_OCP = export_from_WHWEEK(query)
data_Deriv_OCP.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,40656537@WHPROD,N,2024-10-01 01:01:20,WHFLEX,2024-08-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,USD,-219764.12,USD,1,-219764.12,1.00000,-219764.120000,USBFC,None
1,40656537@WHPROD,N,2024-10-01 01:01:20,WHFLEX,2024-08-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,CAD,300000.00,CAD,1,300000.00,0.74118,222353.987548,USBFC,None


In [321]:
data_Deriv_OCP[data_Deriv_OCP.recordId ==1464]


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
72,41551542@WHPROD,N,2024-10-31 00:35:35,WHFLEX,2024-09-30,D,QUANTUM,quantumSwaps,1464,Off-balance,...,3M-6M,USD,-4.737091e+07,USD,1,-4.737091e+07,1.000000,-4.737091e+07,1L00,None
73,41551542@WHPROD,N,2024-10-31 00:35:35,WHFLEX,2024-09-30,D,QUANTUM,quantumSwaps,1464,Off-balance,...,3M-6M,RUB,3.000000e+09,RUB,1,3.000000e+09,0.010753,3.225803e+07,1L00,None


In [288]:
data_Deriv_OCP.recordId = data_Deriv_OCP.recordId.astype(int)
# проверка вхождения в debt
data_Deriv_OCP_work = data_Deriv_OCP.copy()
# проверка вхождения в debt
data_Deriv_OCP_work['in_debt'] = data_Deriv_OCP_work.recordId.isin(data_OCP_export.recordId.unique().tolist())
# data_Deriv_OCP_work = data_Deriv_OCP_work[data_Deriv_OCP_work.in_debt==False]
# Deriv_dealNo = data_Deriv_OCP.recordId.unique().tolist()
# Debt_dealNo = data_OCP_export.recordId.unique().tolist()
# Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
# data_Deriv_OCP_work = data_Deriv_OCP[data_Deriv_OCP.recordId.isin(Deriv_dealNo_new)]
# data_Deriv_OCP_work.shape, data_Deriv_OCP.shape

In [289]:
data_Deriv_OCP_work = data_Deriv_OCP_work[['reportDate','holding','ocpSegment','buCode','salesUnitsId','counterpartyName','recordId','effectiveCurrency','volumeEffectiveCurrency','maturityDateTo', 'in_debt']]
# merge
melt_data_cols = ['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','Currency'] # index col
melt_data.holdingEntity = melt_data.holdingEntity.replace({'EuroChem':'EUROCHEM'}) # replace
# rename
rename_dict = {'reportDate':'positionDate','holding':'holdingEntity','ocpSegment':'Entity_group','buCode':'entity','counterpartyName':'counterparty','recordId':'dealNo','effectiveCurrency':'Currency','volumeEffectiveCurrency':'amount_OCP','maturityDateTo':'maturityDateTo_OCP'} 
data_Deriv_OCP_work = data_Deriv_OCP_work.rename(columns=rename_dict)
# rename кривых названий
data_Deriv_OCP_work['entity'] = merge_Mapping(data_Deriv_OCP_work, 'entity')
# dealNo int
melt_data['dealNo'] = melt_data['dealNo'].astype(int)
data_Deriv_OCP_work['dealNo'] = data_Deriv_OCP_work['dealNo'].astype(int)

Deriv_merge = melt_data.merge(data_Deriv_OCP_work, how='outer', on=melt_data_cols)
# проверка значений на равенство
Deriv_merge = Deriv_merge.fillna(0)
Deriv_merge = is_approximately_equal_for_cols(Deriv_merge, 'amount', 'amount_OCP')

In [290]:
# Deriv_merge.to_excel('d.xlsx')

In [291]:
# assert 2==1

Excel:

In [293]:
### to excel
if print_Debt==True:
    Output_file = f'{str(date.today())}_quDebt_vs_unOCP.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        pd.DataFrame({'manual_check_dealNo':manual_check_dealno_list}, index=range(len(manual_check_dealno_list))).to_excel(writer, index=False, sheet_name='manual_check')
        # First_merge.to_excel(writer, index=False, sheet_name='compare_debt_BU')
        Second_merge.to_excel(writer, index=False, sheet_name='compare_debt_cpty_full')
        Second_merge_work_not_duples.to_excel(writer, index=False, sheet_name='compare_debt_cpty')
        Deriv_merge.to_excel(writer, index=False, sheet_name='compare_deriv')
        data_Debt_work.to_excel(writer, index=False, sheet_name='quDebt')
        data_OCP_export.to_excel(writer, index=False, sheet_name='unOCP')


Ошибки:

Merge counterparty name у меня

In [ ]:
dealno_must_be_not_in_OCP = Second_merge_work_not_duples.loc[Second_merge_work_not_duples.amount_OCP.isna(), 'dealNo'].unique().tolist()
data_OCP_export.loc[data_OCP_export.recordId.isin(dealno_must_be_not_in_OCP), ['introgroupCounterpartyCode', 'counterpartySegment', 'counterpartyCode','counterpartyName']].drop_duplicates()

,introgroupCounterpartyCode,counterpartySegment,counterpartyCode,counterpartyName
7468,None,External,E200,None
7648,None,External,E200,IEFIN


In [ ]:
data_OCP_work.loc[data_OCP_work.dealNo.isin(dealno_must_be_not_in_OCP), ['introgroupCounterpartyCode', 'counterpartySegment', 'counterpartyCode', 'cty_Code', 'Counterparty_Group']].drop_duplicates()

,introgroupCounterpartyCode,counterpartySegment,counterpartyCode,cty_Code,Counterparty_Group
7468,None,External,E200,External,External


In [ ]:
data_Debt_work.loc[data_Debt_work.dealNo.isin(dealno_must_be_not_in_OCP), ['counterparty', 'counterparty_name', 'cty_Code', 'Counterparty_Group']].drop_duplicates()

,counterparty,counterparty_name,cty_Code,Counterparty_Group
6410,IEFIN,IEFIN,IEFIN,SPV
6825,7S00,7S00,7S00,SPV


In [ ]:
data_Debt_work.columns

Index(['##batch_id', '##deleted', '##inserted', '##origin_name', 'flag',
       'actionDate', 'amount', 'eventCurrency', 'amountUSD', 'currentRate',
       'annualInterestUSD', 'amountUSDForMaturity', 'dealNo', 'dealClass1',
       'dealClass2', 'instrumentOwner', 'dealSet', 'facility', 'facilityAlt',
       'loanType', 'facilityAlt2', 'dealTicket', 'holding', 'entity',
       'counterparty', 'counterpartyAlt', 'legDirection', 'dealInternalL0',
       'dealInternalL1', 'issuedLoan', 'year', 'month', 'monthOrdinal',
       'transactionType', 'instrument', 'mmType', 'componentClass',
       'component', 'fixFloat', 'rateFormula', 'basis', 'basisFrequency',
       'currentBaseRate', 'currentMargin', 'rateSetDate', 'cODType',
       'cpartyCountry', 'cpartyRelationL1', 'cpartyType',
       'cpartyDealInternalL0', 'dealExternalL0', 'dealDate', 'settlementDate',
       'currentMaturityDate', 'effectiveDate', 'offerDate', 'entityCountry',
       'entityOwner', 'fxMultiplyToUSD', 'maxFXRateDat